In [6]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display
from bs4 import BeautifulSoup
from openai import OpenAI

In [13]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print('couldn\'t catch API key')

In [55]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self,url):
        self.url =  url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else ''
        if soup.body:
            for irrelevent in soup.body(["script", "style", "img", "input"]):
                irrelevent.decompose()
            self.text = soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = ''

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def website_content(self):
        return (f'The page title is:{self.title}\nThe content is:\n{self.text}')

In [65]:
#a = Website('https://www.nytimes.com/international/')
a = Website('https://edwarddonner.com')
print(a.title)

Home - Edward Donner


In [58]:
#print(a.website_content())

In [62]:
link_system_prompt =
    "You are a brochure assistant. From a list of website links, "
    "select the ones most relevant for describing the company — such as About, Mission, Services, or Careers pages. "
    "Your goal is to choose links that best represent the company's identity for a brochure."


In [76]:
def get_links_user_prompt(website):
    user_prompt = "Here are the links:\n"
    user_prompt += "\n".join(website.links)
    user_prompt+="\nReturn only the most relevant ones in this JSON format:\n"
    user_prompt+="""{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
    }"""
    return user_prompt

In [78]:
print(get_links_user_prompt(a))

Here are the links:
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-en